In [13]:
import pandas as pd
import numpy as np
import os
import requests
import locale

# Set the locale to the user's default (or a specific one like 'en_US.UTF-8')
locale.setlocale(locale.LC_ALL, '')


# Define the file path and URL
file_path = 'portfolio_file.csv'
url = os.getenv('RESP_FOR_EXPORT_URL')

# Check if the file exists locally
if not os.path.exists(file_path):
    print(f"{file_path} not found. Downloading from the URL...")
    response = requests.get(url)
    if response.status_code == 200:
        # Save the file
        with open(file_path, 'wb') as f:
            f.write(response.content)
        print(f"File downloaded and saved as {file_path}.")
    else:
        print(f"Failed to download the file. Status code: {response.status_code}")
        exit()
# Load the file
fundsneeded=5300
df = pd.read_csv(file_path)

print(df)

      Stock   QTY  Price  Dividend
0   TSE:BCE   200  33.90      3.99
1   TSE:CNQ   740  46.80      2.25
2   TSE:ENB  3044  63.06      3.66
3   TSE:PSA     0  50.05       NaN
4   TSE:TRP   300  68.39      3.52
5  TSE:SOBO    74  33.40      2.76


**I need to remove rows where the qty is zero**

In [9]:
df = df[df['QTY'] != 0] 

Now I need to add the columns I care about for my sort

In [54]:
df['required'] = np.ceil(fundsneeded / df['Price'])
df['incomeloss']=np.ceil(df['required'] * df['Dividend'])
df = df.sort_values(by='incomeloss', ascending=True)
print(df)

      Stock   QTY  Price  Dividend  required  incomeloss
1   TSE:CNQ   740  46.80      2.25     114.0       257.0
4   TSE:TRP   300  68.39      3.52      78.0       275.0
2   TSE:ENB  3044  63.06      3.66      85.0       312.0
5  TSE:SOBO    74  33.40      2.76     159.0       439.0
0   TSE:BCE   200  33.90      3.99     157.0       627.0


In [14]:
first_row = df.iloc[0]  # Get the first row as a Series


# Access specific values in the first row
price = first_row['Price']
req_qty = first_row['required']
proceeds = price * req_qty

print(f"CREATE Order:  Sell {first_row['Stock']}:{req_qty} @ {price} for proceeds of {locale.currency(proceeds, grouping=True)} ")

KeyError: 'required'